In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

In [ ]:
# 加载训练数据
mnist_train = torchvision.datasets.FashionMNIST(
    root='~/Datasets/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

In [ ]:
# 加载测试数据
mnist_test = torchvision.datasets.FashionMNIST(
    root='~/Datasets/FashionMNIST',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [ ]:
def get_fashion_mnist_labels(labels):
    '''返回Fashion-MNIST数据集的文本标签'''
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [ ]:
def show_fashion_mnist(images, labels):
    '''展示图像和标签'''
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((28, 28)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
print(mnist_train.data.shape)
'''
60000个样本
每个样本大小28*28=784
'''
print(len(mnist_train.targets.unique()))
'''
10个标签
'''

In [ ]:
class SoftMaxRegression:
    def __init__(self):
        # 初始化模型参数
        self.num_inputs = 784
        self.num_outputs = 10

        self.W = torch.normal(0, 0.01, size=(self.num_inputs, self.num_outputs), requires_grad=True) #每一个像素点的回归系数
        self.b = torch.zeros(self.num_outputs, requires_grad=True) #偏置项

    def soft_max(self, X):
        """   计算softmax
        输入: X, 其中size=(n, 10)

        对每一行X计算softmax

        输出: softmax(X), 其中size=(n, 10)

        注意不要用for实现!!!
        """
        return 0

    def net(self, X):
        """定义模型"""
        X = X.view(-1, self.num_inputs)
        return self.soft_max(torch.mm(X, self.W) + self.b)

    def one_hot_encoder(self, y):
        """独热编码"""
        a = torch.zeros((len(y), self.num_outputs))
        for idx, i in enumerate(y):
            a[idx, int(i)] = 1
        return a
    
    def cross_entropy(self, y_hat, y):
        y_pred_enc = self.one_hot_encoder(y)
        """   损失函数
        输入: 
            y_hat, 模型预测概率分布, 其中size=(n, 10)
            y, 真实标签, 其中size=(n, 1)
        
        先对y进行独热编码转为y_pred_enc使其size=(n, 10), 然后计算y_pred_enc与y_hat的交叉熵损失

        输出: y_pred_enc与y_hat的 n 个交叉熵损失值

        注意不要用for实现!!!
        """
        return 0

    def accuracy(self, y_hat, y):
        """   统计预测正确的数量
        输入: 
            y_hat, 模型预测概率分布, 其中size=(n, 10)
            y, 真实标签, 其中size=(n, 1)
        
        根据概率分布y_hat的标签, 然后和真实标签对比统计预测正确的数量

        输出: correct_count 预测正确的数量

        注意不要用for实现!!!
        """
        return 0

    def evaluate_accuracy(self, net, data_iter):
        """计算在指定数据集上模型的精度"""
        correct_count, total_count = 0, 0
        with torch.no_grad():
            for X, y in data_iter:
                correct_count += self.accuracy(net(X), y)
                total_count += y.numel()
        return correct_count / total_count

    def train_epoch(self, net, train_iter, loss):
        """训练模型的一个迭代周期"""
        total_loss, correct_count, total_count = 0, 0, 0
        lr = 0.1
        for X, y in train_iter:
            y_hat = net(X)
            # 计算损失函数
            l = loss(y_hat, y)
            # 求梯度
            l.mean().backward()
            # 更新训练参数
            self.W.data -= lr * self.W.grad
            self.b.data -= lr * self.b.grad
            # 梯度清零
            self.W.grad.data.zero_()
            self.b.grad.data.zero_()
            #计算指标
            total_loss += l.sum().item()
            correct_count += self.accuracy(y_hat, y)
            total_count += y.numel()
        # 返回训练损失和训练精度
        return total_loss / total_count, correct_count / total_count

    def train(self, net, train_iter, test_iter, loss, num_epochs):
        """训练模型"""
        for epoch in range(num_epochs):
            train_metrics = self.train_epoch(net, train_iter, loss)
            test_acc = self.evaluate_accuracy(net, test_iter)
            train_loss, train_acc = train_metrics
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss}, train_acc: {train_acc}, test_acc: {test_acc}')
            assert train_acc <= 1 and train_acc > 0.7, train_acc
            assert test_acc <= 1 and test_acc > 0.7, test_acc

    def predict(self, net, test_iter, n=10):
        """预测标签"""
        X, y = next(iter(test_iter))
        trues = get_fashion_mnist_labels(y)
        preds = get_fashion_mnist_labels(net(X).argmax(axis=1))
        titles = [true + '\n' + pred for true, pred in zip(trues, preds)]# 真实标签与预测标签
        show_fashion_mnist(X[0:n], titles[0:n])# 展示图像及其标签[trues, preds]

    def run(self,train_iter,test_iter,num_epochs = 1):
        self.train(self.net, train_iter, test_iter, self.cross_entropy, num_epochs)
        self.predict(self.net, test_iter)

In [ ]:
batch_size = 256
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=4)
SMR = SoftMaxRegression()
SMR.run(train_iter,test_iter)